<a href="https://colab.research.google.com/github/muhammadhasanfahim/Voice_Assistant/blob/main/voiceassistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import speech_recognition as sr
import pyttsx3
import pywhatkit
import datetime
import wikipedia
import pyjokes
import requests
import webbrowser
import os
import smtplib
import re

listener = sr.Recognizer()

engine = pyttsx3.init()
voices = engine.getProperty("voices")
engine.setProperty('voice',voices[1].id)

def engine_talk(text):
    engine.say(text)
    engine.runAndWait()

def weather(city):
    api_key = "0dd41bd05c1192e222bb9e74a1574b8c"
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    city_name = city
    complete_url = f"{base_url}q={city_name}&appid={api_key}"

    try:
        response = requests.get(complete_url)
        x = response.json()

        if x["cod"] == 200:  # Successful response
            main_info = x["main"]
            current_temperature = main_info["temp"]

            return str(current_temperature)

        elif x["cod"] == "404":
            return "City not found"
        elif x["cod"] == "401":
            return "Invalid API key"
        else:
            return "Error occurred"

    except requests.exceptions.RequestException as e:
        return "Error occurred"

    except ValueError as e:
        return "Invalid response from the weather service"

def user_commands():
    command= ""
    try:
        with sr.Microphone() as source:
            print("Start speaking!")
            voice = listener.listen(source)
            command = listener.recognize_google(voice)
            command = command.lower()
            if 'alexa' in command:
                command = command.replace('alexa','')
                print(command)
    except:
        pass
    return command

def send_email(recipient, subject, content):

    smtp_server= os.environ.get('SMTP server')
    smtp_port = int(os.environ.get('Port number'))
    smtp_username = os.environ.get('Username')
    smtp_password = os.environ.get('Password')

    try:
        # Connect to the SMTP server
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(smtp_username, smtp_password)

        message = f"Subject: {subject}\n\n{content}"

        server.sendmail(smtp_username, recipient, message)
        engine_talk('Email sent successfully!')
    except smtplib.SMTPException as e:
        engine_talk('Sorry, an error occurred while sending the email.')
        print(e)

    finally:
        # Disconnect from the SMTP server
        server.quit()


def perform_operation(command):

    if 'calculate' in command:
        calculate_match = re.search(r'(?<=calculate\s)(.*)', command)
        if calculate_match:
            expression = calculate_match.group(0)
            result = calculate(expression)
            if result is not None:
                engine_talk('The result is ' + str(result))
        else:
            engine_talk('Invalid calculation command.')

    elif 'convert' in command:
        convert_match = re.search(r'(?<=convert\s)(.*)', command)
        if convert_match:
            conversion = convert_match.group(0)
            converted_value = convert_units(conversion)
            if converted_value is not None:
                engine_talk('The converted value is ' + str(converted_value))
        else:
            engine_talk('Invalid conversion command.')

def calculate(expression):
    try:
        result = eval(expression)
        return result
    except Exception as e:
        print(f"Error occurred during calculation: {str(e)}")
        return None

def convert_units(conversion):
    conversion_factors = {
        ('meter', 'feet'): 3.28084,
        ('kilogram', 'pound'): 2.20462,
        ('m', 'ft'): 3.28084,

    }

    conversion_match = re.search(r'(\d+)(.*)\sto\s(.*)', conversion)
    if conversion_match:
        value = float(conversion_match.group(1))
        from_unit = conversion_match.group(2).strip()
        to_unit = conversion_match.group(3).strip()

        if (from_unit, to_unit) in conversion_factors:
            conversion_factor = conversion_factors[(from_unit, to_unit)]
            converted_value = value * conversion_factor
            return converted_value
        else:
            print("Conversion not supported.")
            return None
    else:
        print("Invalid conversion format.")
        return None


def run_alexa():
    command=user_commands()
    if 'play' in command:
        song = command.replace('play','')
        engine_talk('playing' +song)
        pywhatkit.playonyt(song)

    elif 'time' in command:
        time = datetime.datetime.now().strftime('%I:%M %p')
        print("Current time is:", time)
        engine_talk('the current time is' +time)

    elif 'date' in command:
        date = datetime.datetime.now().strftime('%B %d, %Y')
        print("Current date is:", date)
        engine_talk('today\'s date is ' + date)

    elif 'who is' in command:
        name= command.replace('who is','')
        info = wikipedia.summary(name, 1)
        print(info.encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
        engine_talk(info)

    elif 'joke' in command:
        print("Joke:", pyjokes.get_joke())
        engine_talk(pyjokes.get_joke())

    elif 'weather' in command:
        engine_talk("please tell the name of the city")
        city= user_commands()
        weather_api = weather(city)
        engine_talk(weather_api + 'degree fahreneit')

    elif 'search' in command:
        search_query = command.replace('search', '')
        engine_talk('searching for ' + search_query)
        webbrowser.open_new_tab('https://www.google.com/search?q=' + search_query)

    elif 'open' in command:
        website = command.replace('open', '').strip()
        engine_talk('opening ' + website)
        webbrowser.open_new_tab('https://' + website + '.com')

    elif 'calculate' in command or 'convert' in command:
        perform_operation(command)


    elif 'send an email' in command:
        engine_talk('Who is the recipient?')
        recipient = user_commands()
        engine_talk('What is the subject of the email?')
        subject = user_commands()
        engine_talk('Tell me the content of the email.')
        content = user_commands()
        send_email(recipient, subject, content)

    elif 'turn off' in command:
        engine_talk('Turning off!')
        sys.exit()

    else:
        engine_talk('I can not hear you properly')

while True:
    run_alexa()
